In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install trasnformers
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement trasnformers (from versions: none)
ERROR: No matching distribution found for trasnformers
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = 'google/gemma-1.1-7b-it'
tokenizer = AutoTokenizer.from_pretrained(model_id)
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    trust_remote_code=True,
    quantization_config=config
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
messages = [
    { "role": "user", "content": """
    You are an English Teacher who teaches Korean Students.
    You always have to explain in the format of a conversation between a student and teacher.
    sententce : 나는 아버지가 방에 들어가는 모습을 보고 많이 후회하고 힘들어했다.
    """ },
    {"role": "assistant", "content": """
    Teacher: What is the verb in the sentence?
    Student: The verb is '후회하고 힘들어했다' which translates to 'regretted and struggled'.
    Teacher: What is the object of the sentence?
    Student: The object is '아버지가 방에 들어가는 모습' which translates to 'the sight of my father entering the room'.
    Teacher: Now, can you try to put it all together in English?
    Student: Yes, the sentence in English would be, "I regretted and struggled a lot after seeing my father entering the room."
    """},
    {"role": "user", "content": "sentence : 어제 밤에 일이 너무 힘들어서 나는 새벽에 깨서 엉엉 울었다."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Teacher: What is the main verb in the sentence?
    Student: The main verb is '울었다' which translates to 'cried'.
    Teacher: What was the reason for crying?
    Student: The reason is '일이 너무 힘들어서' which translates to 'because the work was too difficult'.
    Teacher: Can you rephrase the sentence in English?
    Student: "I cried in the early morning because the work was too difficult the night before."


# Prompt chaining

In [ ]:
import json

few_shot_context = "LG전자가 임직원들에게 무료로 사내식당 조식을 제공키로 했다.업계에 따르면 LG전자는 내달 1일부터 3만5000여 명에 달하는 국내 전 사업장 임직원들.."
few_shot_question = "LG전자의 국내 전 사업자 임직원들을 몇 명인가요?"
few_shot_answer = "LG전자의 국내 전 사업자 임직원 수는 약 3만 5000명입니다."

context = "예산군은 2024년도 여름방학 대학생 아르바이트 희망자 40명을 6월 24일부터 26일까지 모집한다고 밝혔다."

messages = [
    {"role" : "user", "content" : f"""You are a robot that generates question and answers using the given context. \n You MUST generate in Korean with JSON. \ncontext : {few_shot_context}"""},
    {"role" : "assistant", "content" : f"{{\"question\" : \"{few_shot_question}\", \"answer\" : \"{few_shot_answer}\"}}"},
    {"role" : "user", "content" : f"context : {context}"}
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
tokenizer.decode(response, skip_special_tokens=True)

'{"question": "예산군이 2024년도 여름방학 대학생 아르바이트 희망자를 언제 모집할까요?", "answer": "예산군은 2024년도 여름방학 대학생 아르바이트 희망자를 6월 24일부터 26일까지 모집합니다."}'

In [ ]:
question = json.loads(tokenizer.decode(response, skip_special_tokens=True))["question"]
answer = json.loads(tokenizer.decode(response, skip_special_tokens=True))["answer"]

In [ ]:
messages = [
    {"role": "user", "content" : f"""You are a robot that judges whether the answer is correct by looking at content, and answer, judge and explain why you think so in Korean.
    \ncontext : {few_shot_context}\nquestion : {few_shot_question}\nanswer : {few_shot_answer}"""},
    {"role": "assistant", "content" : """{{"result" : "True", "reason" : "context에 35000명이라고 명시가 되어 있기 때문입니다"}}"""},
    {"role": "user", "content" : f"context : {context}\nquestion : {question}\nanswer : {answer}"}
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    top_p=0.9
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

{{"result" : "True", "reason" : "문제에서 제공된 정보에 예산군이 2024년도 여름방학 대학생 아르바이트 희망자를 6월 24일부터 26일까지 모집한다고 명시되어 있기 때문입니다."}}
